In [1]:
import os
import pandas as pd
import numpy as np
os.chdir('C:/Users/tm/Downloads/utas/thesis/chapter1/tasman/poems')
from scipy.optimize import curve_fit

In [2]:
#max capacity assuming one devil per sq km
valid=pd.read_csv('validation.csv')
K_max=len(valid[valid['target']==1])

In [3]:
#population size via cormack jolly seber
population=pd.read_csv('population.csv')

In [4]:
#leslie matrix
pre=[[0.        , 0.53061224, 0.76666667, 1.38709677, 0.88461538,
        0.25     ],
       [0.9521518 , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.9241624 , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.9043036 , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.88889992, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.87631419,
        0.        ]]

post=[[0.07407407, 0.212     , 1.59803922, 1.71052632, 1.55      ,
        0.64772727],
       [0.73785791, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.58451461, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.47571581, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.39132491, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.32237252,
        0.        ]]

In [5]:
#compute density dependant function discount factor
N1=population['population'].loc[1:4].mean()
N2=population['population'].loc[6:10].mean()

factor1=(1-N1/K_max)
factor2=(1-N2/K_max)

In [6]:
#simple density dependant function
def density_func(x,baseline):
    return [x[0]*baseline,x[1]*baseline]

#using mle to estimate
def get_dens_params(x,result):
    popt,pcov=curve_fit(density_func,x,result,p0=(0.5))
    return popt

In [7]:
#estimate fecundity rate
fecundity_rate=[]
for ind,val in enumerate(pre[0]):
    if ind==0:
        continue
    
    fecundity_rate.append(get_dens_params([factor1,factor2],
                                              [post[0][ind],pre[0][ind]])[0])

In [8]:
#simple density dependant function with disease induced mortality
def disease_func(x,b1,b2,b3,b4,b5,delta):    
    baseline=[b1,b2,b3,b4,b5]
    dens_result=np.r_[np.multiply(baseline,x[0]),np.multiply(baseline,x[1])]
    result=np.subtract(dens_result,delta)
    return result.tolist()

#using mle to estimate
def get_disease_params(x,result):
    popt,pcov=curve_fit(disease_func,x,result,
                        bounds=(0, [1.,1.,1.,1.,1.,1.,]),
                       )
    return popt

In [9]:
#get survival rate
diag1=np.diag(np.array(post)[1:,:-1])
diag2=np.diag(np.array(pre)[1:,:-1])

In [10]:
#get result
ans=get_disease_params([factor1,factor2],np.r_[diag1,diag2].tolist())

In [11]:
#create leslie matrix
baseline_diag=np.c_[np.diag(ans[:len(diag1)]),np.zeros((len(diag1),1))]

baseline_first_row=np.array([0]+fecundity_rate).reshape(1,-1)

np.r_[baseline_first_row,baseline_diag]

array([[0.        , 0.4585334 , 1.38793539, 1.84804797, 1.43641906,
        0.52369967],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.95675756, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.8829525 , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.82570485, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.77893011,
        0.        ]])

In [12]:
#disease induced mortality
ans[-1]

0.03099668618478338

In [16]:
#dominant eigenval
np.linalg.eigvals(np.r_[baseline_first_row,baseline_diag])[0]-1

(0.5269428792468884+0j)